In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import re

# Generate dataset

names = ["condensed_2010.json","condensed_2011.json","condensed_2012.json","condensed_2013.json",
         "condensed_2014.json", "condensed_2015.json","condensed_2016.json","condensed_2017.json","condensed_2018.json",
         "trump2019_26_03.json"]

data = pd.read_json("condensed_2009.json")
for element in names:
    data1 = pd.read_json(element)
    print(len(data1),element)
    data = data.append(data1,sort=True)
data = data.reset_index(drop=True)


#pd.to_datetime(stamps, format="%Y-%m-%d %H:%M:%S").sort_values()

data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data.to_json("orderedTwitterArchive.json")
data.to_csv("orderedTwitterArchive.csv")
data

# Exploration

In [2]:
data = pd.read_json("orderedTwitterArchive.json")
data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data

,created_at,favorite_count,id_str,in_reply_to_user_id_str,is_retweet,retweet_count,source,text
0,2009-05-04 18:54:25,202,1698308935,NaN,False,253,Twitter Web Client,Be sure to tune in and watch Donald Trump on L...
1,2009-05-05 01:00:10,3,1701461182,NaN,False,2,Twitter Web Client,Donald Trump will be appearing on The View tom...
2,2009-05-08 13:38:08,2,1737479987,NaN,False,3,Twitter Web Client,Donald Trump reads Top Ten Financial Tips on L...
3,2009-05-08 20:40:15,27,1741160716,NaN,False,8,Twitter Web Client,New Blog Post: Celebrity Apprentice Finale and...
4,2009-05-12 14:07:28,1950,1773561338,NaN,False,1421,Twitter Web Client,"""My persona will never be that of a wallflower..."
5,2009-05-12 19:21:55,13,1776419923,NaN,False,10,Twitter Web Client,"Miss USA Tara Conner will not be fired - ""I've..."
6,2009-05-13 17:38:28,10,1786560616,NaN,False,11,Twitter Web Client,Listen to an interview with Donald Trump discu...
7,2009-05-14 16:30:40,6,1796477499,NaN,False,3,Twitter Web Client,"""Strive for wholeness and keep your sense of w..."
8,2009-05-15 14:13:13,2,1806258917,NaN,False,1,Twitter Web Client,"Enter the ""Think Like A Champion"" signed book ..."
9,2009-05-16 22:22:45,5,1820624395,NaN,False,3,Twitter Web Client,"""When the achiever achieves, it's not a platea..."


In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax = sns.lineplot(x=data.index, y="favorite_count", data=data) 

In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax = sns.lineplot(x=data.index, y="retweet_count", data=data) 

In [3]:
data["text"][2005]


"China has copied our military's F-22 Raptor design http://t.co/Oqi65Leb We should offset their theft from our debt."

In [4]:
#check tags
ats = dict()
for idx,row in data.iterrows():
    l = re.findall("^@[A-z]*",row["text"])
    for el in l:
        if el not in ats:
            ats[el] = 1
        else:
            ats[el] = ats[el]+1
ats

{'@shouldtrumprun': 1,
 '@danmcisaac': 1,
 '@oreillyfactor': 2,
 '@TheRealMissUSA': 1,
 '@GolfMagazine': 1,
 '@johnboehner': 6,
 '@Obama': 1,
 '@macmiller': 1,
 '@BarackObama': 53,
 '@marcorubio': 2,
 '@KarlRove': 4,
 '@Johnboehner': 2,
 '@MacMiller': 1,
 '@RonPaul': 1,
 '@IvankaTrump': 3,
 '@EricTrump': 1,
 '@JonHuntsman': 3,
 '@VPBiden': 1,
 '@AmandaKnox': 1,
 '@USGovernment': 1,
 '@THeCelebrityApprentice': 1,
 '@Israel': 1,
 '@Amazon': 1,
 '@TrumpCollection': 1,
 '@nathanfuchs': 1,
 '@IanBQueen': 1,
 '@Miss_Drax': 1,
 '@bats': 1,
 '@GrimeyGatsby': 1,
 '@stass': 2,
 '@fatmaninaspeedo': 1,
 '@npbraden': 1,
 '@Misterglorious': 1,
 '@JLukeRobinson': 1,
 '@bonniebell': 1,
 '@cheatingdeam': 1,
 '@WyattBrian': 1,
 '@MittRomney': 12,
 '@The_Joey': 1,
 '@HangersMensLine': 1,
 '@atlthrasher': 1,
 '@kingdelrosario': 1,
 '@fox_trot_': 1,
 '@joeyhatcher': 1,
 '@jtatsuno': 2,
 '@stecbeck': 1,
 '@dansalbreiter': 1,
 '@MemoInc': 1,
 '@JLin': 1,
 '@CelebApprentice': 1,
 '@SachaBaronCohen': 1,
 '@Nic

In [17]:
s = pd.Series(ats)
s

@shouldtrumprun             1
@danmcisaac                 1
@oreillyfactor              2
@TheRealMissUSA             1
@GolfMagazine               1
@johnboehner                6
@Obama                      1
@macmiller                  1
@BarackObama               53
@marcorubio                 2
@KarlRove                   4
@Johnboehner                2
@MacMiller                  1
@RonPaul                    1
@IvankaTrump                3
@EricTrump                  1
@JonHuntsman                3
@VPBiden                    1
@AmandaKnox                 1
@USGovernment               1
@THeCelebrityApprentice     1
@Israel                     1
@Amazon                     1
@TrumpCollection            1
@nathanfuchs                1
@IanBQueen                  1
@Miss_Drax                  1
@bats                       1
@GrimeyGatsby               1
@stass                      2
                           ..
@LukeBryanOnline            1
@tiberias                   1
@TheRevAl 

In [5]:
#check hastags
hashtags = dict()
for idx,row in data.iterrows():
    l = re.findall("^#[A-z]*",row["text"])
    for el in l:
        if el not in ats:
            hashtags[el] = 1
        else:
            hashtags[el] = hashtags[el]+1
hashtags
#usa pochissimo gli hashtag

{'#trumpvlog': 1,
 '#timetogettough': 1,
 '#TimeToGetTough': 1,
 '#badratings': 1,
 '#noratings': 1,
 '#SuperBowl': 1,
 '#CelebApprentice': 1,
 '#MidasTouch': 1,
 '#CelebrityApprentice': 1,
 '#SuccessByTrump': 1,
 '#sweepstweet': 1,
 '#SweepsTweet': 1,
 '#FullRepeal': 1,
 '#BARACKTAX': 1,
 '#TrumpTower': 1,
 '#TRUMP': 1,
 '#TBT': 1,
 '#TrumpVine': 1,
 '#TrumpAdvice': 1,
 '#FlashbackFriday': 1,
 '#TheView': 1,
 '#': 1,
 '#HasJustineLandedYet': 1,
 '#TheArsenioHallShow': 1,
 '#IceBucketChallenge': 1,
 '#TrumpVlog': 1,
 '#IndianaJones': 1,
 '#MakeAmericaGreatAgain': 1,
 '#MayThe': 1,
 '#CNNDebate': 1,
 '#AskTrump': 1,
 '#DemDebate': 1,
 '#ChrisWallace': 1,
 '#TrumpTODAY': 1,
 '#ICYMI': 1,
 '#TeamTrump': 1,
 '#TheRemembranceProject': 1,
 '#GOPDebate': 1,
 '#ISIS': 1,
 '#HappyNewYearAmerica': 1,
 '#VoteTrump': 1,
 '#FoxNews': 1,
 '#Trump': 1,
 '#TedCruz': 1,
 '#IACaucus': 1,
 '#VoteTrumpNH': 1,
 '#VoteTrumpKS': 1,
 '#CaucusForTrump': 1,
 '#VoteTrumpMS': 1,
 '#VoteTrumpMI': 1,
 '#VoteTrumpHI

In [10]:
#remove hashtags, useless
for idx,row in data.iterrows():
    re.sub("^#[A-z]*","",row["text"])
#usa pochissimo gli hashtag

genereate new column with modded text, remove stopwords

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["modded_text"] = data["text"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
data

In [ ]:
#remove punctuation
data['modded_text'] = data["modded_text"].str.replace("[^\w\s]",'')

#stemming
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()

stemmedDataset = pd.DataFrame()
data1 = data

stemDescriptions = []
for idx, row in data1.iterrows():
    desc = ""
    for el in row["text"].split():
        desc = desc + stemmer.stem(el) + " "
    stemDescriptions.append(desc)
    
data1["modded_text"] = stemDescriptions


#unique words
unique = pd.Series(' '.join(data1['modded_text']).split()).value_counts()
print(len(unique))
#devi rimuovere le cose con valore meno di tot e più di tot

In [ ]:
data1["modded_text"]

In [ ]:
unique.mean()


In [ ]:
unfreq = []
for el in unique.index:
    if unique[el] <= 5:
        unfreq.append(el)
len(unfreq)

# idee

NLP:
    - pos tagging per contare numero di aggettivi 
    - qualcosa per vedere se aggettivi sono positivi o negativi (direttamente sentiment analysis)?
    - rimuovere gli @ vs tenere solo i più frequenti 
    - rimuovere i link
    - pulire gli hashtag? 
    - word2vec distance per avere simili tweet assieme nello spazio vettoriale?
    - ridurre parole presenti? 
    - tfidf?
    
Processo:
    - clustering interno all'anno
    - clustering generale 
    - clustering con/senza sentiment meaning